Befor running the script pass the location of the .txt file to source_text parameter.   
Run the code and finally in the testing area, input the sentence for which the model predicts the output.   

In [1]:
source_text='/content/gdrive/MyDrive/science_fiction/internet_archive_scifi_v3.txt'

In [2]:
from google.colab import drive
drive.mount('/content/gdrive/')

Mounted at /content/gdrive/


In [3]:
# importing required libraries
import string
import numpy as np
import tensorflow as tf
from keras.preprocessing.text import Tokenizer
from pickle import dump
from keras.preprocessing.text import Tokenizer
from tensorflow.keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Embedding
from keras.layers import Dropout
from numpy import array
from pickle import dump
from random import randint
from pickle import load
from keras.models import load_model
from keras.utils import pad_sequences
import keras

In [4]:
# Loading text into memory
file= open(source_text, 'r')
data= file.read()
file.close()
print(data[:100])

MARCH # All Stories New and Complete Publisher Editor IF is published bi-monthly by Quinn Publishing


In [5]:
data=data[5000:1000000]

In [6]:
# Cleaning text
def clean_data(data):
    # replace '--' with a space
    data= data.replace('--', ' ')
    #splitting into tokens by space
    words= data.split()
    #removing punctuations
    words= [word.translate(str.maketrans('', '', string.punctuation)) for word in words]
    #removing non-aplhabetic tokens
    words= [word for word in words if word.isalpha()]
    # lower-case
    tokens= [word.lower() for word in words]
    return tokens

tokens= clean_data(data)
print(len(tokens))
print(len(set(tokens)))

177240
15101


In [7]:
# creating sequences from tokens
length= 11 #length of sequence()
sequences= list()
for i in range(length, len(tokens)):
    seq= tokens[i-length:i]
    line= ' '.join(seq)
    sequences.append(line)
print(len(sequences))

177229


In [8]:
!mkdir /data/

In [9]:
# saving into file
data = '\n'.join(sequences)
file = open('/data/scify_preprocessed.txt', 'w')
file.write(data)
file.close()

In [10]:
# loading scify_preprocessed.txt file
file= open('/data/scify_preprocessed.txt', 'r')
doc= file.read()
file.close()
lines= doc.split('\n')

In [11]:
#cleaning the memory
import gc
del data
gc.collect()

0

In [12]:
# Encoding Sequences
tokenizer = Tokenizer()
tokenizer.fit_on_texts(lines)
sequences = tokenizer.texts_to_sequences(lines)
#padded_sequences = pad_sequences(sequences, padding='post')

In [13]:
# Size of vocabolary
vocab_size = len(tokenizer.word_index) + 1
print(vocab_size)

15101


In [14]:
# Separating into input, output sequences
sequences = array(sequences)# converting to array
x, y = sequences[:,:-1], sequences[:,-1]

In [15]:
# One-hot encoding output
y = to_categorical(y, num_classes=vocab_size)#vocab_size)
seq_length = x.shape[1]
print(seq_length)

10


In [ ]:
#test code
#vocab_size=10000

In [ ]:
# Builduing LSTM model
model = Sequential()
model.add(Embedding(vocab_size, 10, input_length=seq_length))
model.add(LSTM(32, return_sequences=True))
model.add(LSTM(64))
model.add(Dense(64, activation='relu'))
model.add(Dense(128, activation='relu'))
model.add(Dense(vocab_size, activation='softmax'))
print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 10, 10)            151010    
                                                                 
 lstm (LSTM)                 (None, 10, 32)            5504      
                                                                 
 lstm_1 (LSTM)               (None, 64)                24832     
                                                                 
 dense (Dense)               (None, 64)                4160      
                                                                 
 dense_1 (Dense)             (None, 128)               8320      
                                                                 
 dense_2 (Dense)             (None, 15101)             1948029   
                                                                 
Total params: 2,141,855
Trainable params: 2,141,855
Non-

In [ ]:
callback = tf.keras.callbacks.EarlyStopping(monitor='accuracy', patience=3)

In [ ]:
# Compiling and fitting model
model.compile(loss='categorical_crossentropy', optimizer='Adam', metrics=['accuracy'])

history= model.fit(x, y, batch_size=200, epochs=400,callbacks=[callback],verbose=1)

In [ ]:
#igone this part as it used for re training the already trained model bu the performance was not increasing
history= model.fit(x, y, batch_size=100, epochs=100,callbacks=[callback],verbose=1)

Epoch 1/100
1773/1773 [==============================] - 38s 17ms/step - loss: 3.3090 - accuracy: 0.2935
Epoch 2/100
1773/1773 [==============================] - 19s 11ms/step - loss: 3.2895 - accuracy: 0.2974
Epoch 3/100
1773/1773 [==============================] - 19s 11ms/step - loss: 3.2916 - accuracy: 0.2965
Epoch 4/100
1773/1773 [==============================] - 19s 11ms/step - loss: 3.2846 - accuracy: 0.2985
Epoch 5/100
1773/1773 [==============================] - 18s 10ms/step - loss: 3.2760 - accuracy: 0.2996
Epoch 6/100
1773/1773 [==============================] - 19s 11ms/step - loss: 3.2691 - accuracy: 0.3009
Epoch 7/100
1773/1773 [==============================] - 19s 10ms/step - loss: 3.2596 - accuracy: 0.3026
Epoch 8/100
1773/1773 [==============================] - 18s 10ms/step - loss: 3.2585 - accuracy: 0.3027
Epoch 9/100
1773/1773 [==============================] - 18s 10ms/step - loss: 3.2498 - accuracy: 0.3043
Epoch 10/100
1773/1773 [==============================]

In [ ]:
#igone this part as it used for re training the already trained model bu the performance was not increasing
history= model.fit(x, y, batch_size=150, epochs=150,verbose=1)

Epoch 1/150
1182/1182 [==============================] - 32s 23ms/step - loss: 2.8866 - accuracy: 0.3664
Epoch 2/150
1182/1182 [==============================] - 15s 13ms/step - loss: 2.8860 - accuracy: 0.3655
Epoch 3/150
1182/1182 [==============================] - 14s 12ms/step - loss: 2.8773 - accuracy: 0.3678
Epoch 4/150
1182/1182 [==============================] - 14s 12ms/step - loss: 2.8745 - accuracy: 0.3685
Epoch 5/150
1182/1182 [==============================] - 13s 11ms/step - loss: 2.8758 - accuracy: 0.3680
Epoch 6/150
1182/1182 [==============================] - 14s 12ms/step - loss: 2.8768 - accuracy: 0.3671
Epoch 7/150
1182/1182 [==============================] - 14s 12ms/step - loss: 2.8656 - accuracy: 0.3697
Epoch 8/150
1182/1182 [==============================] - 14s 12ms/step - loss: 2.8680 - accuracy: 0.3700
Epoch 9/150
1182/1182 [==============================] - 14s 12ms/step - loss: 2.8716 - accuracy: 0.3684
Epoch 10/150
1182/1182 [==============================]

In [ ]:
#igone this part as it used for re training the already trained model bu the performance was not increasing
history= model.fit(x, y, batch_size=64, epochs=100,verbose=1)

Epoch 1/400
887/887 [==============================] - 31s 29ms/step - loss: 7.1738 - accuracy: 0.0582
Epoch 2/400
887/887 [==============================] - 13s 14ms/step - loss: 6.8411 - accuracy: 0.0590
Epoch 3/400
887/887 [==============================] - 11s 13ms/step - loss: 6.5955 - accuracy: 0.0692
Epoch 4/400
887/887 [==============================] - 11s 13ms/step - loss: 6.3877 - accuracy: 0.0769
Epoch 5/400
887/887 [==============================] - 11s 12ms/step - loss: 6.2277 - accuracy: 0.0837
Epoch 6/400
887/887 [==============================] - 11s 12ms/step - loss: 6.0838 - accuracy: 0.0905
Epoch 7/400
887/887 [==============================] - 11s 12ms/step - loss: 5.9448 - accuracy: 0.0975
Epoch 8/400
887/887 [==============================] - 11s 12ms/step - loss: 5.8190 - accuracy: 0.1032
Epoch 9/400
887/887 [==============================] - 11s 12ms/step - loss: 5.7039 - accuracy: 0.1078
Epoch 10/400
887/887 [==============================] - 11s 12ms/step - l

In [ ]:
model=load_model('/content/gdrive/MyDrive/model_text_1.h5')

In [ ]:
# Saving the trained model
model.save('/content/gdrive/MyDrive/model_text_1.h5')

In [ ]:
# saving tokeniser
dump(tokenizer, open('/content/gdrive/MyDrive/science_fiction/tokenizer.pkl', 'wb')) #provide the location to save the tokenizer

### Testing on Unknown text/ words

In [16]:
model_load=load_model('/content/gdrive/MyDrive/210409183_textprediction_model.h5')#loadig model

In [ ]:
# Loading the model and tokeniser
#model = load_model("/content/gdrive/MyDrive/science_fiction/model_1.h5")
tokenizer = load(open("/content/gdrive/MyDrive/science_fiction/tokenizer.pkl", 'rb')) #loading tokenizer

In [17]:
# test words
test_text= '''the people are scared because
'''

In [18]:
print(test_text)

the people are scared because



In [19]:
# Cleaning text
def clean_text_1(text):
    # replace '--' with a space
    text= text.replace('--', ' ')
    #splitting into words by space
    tokens= text.split()
    # punctuations removal
    tokens= [token.translate(str.maketrans('', '', string.punctuation)) for token in tokens]
    #removing non-aplhabetic tokens
    tokens= [word for word in tokens if word.isalpha()]
    # lower-case
    tokens= [word.lower() for word in tokens]
    return tokens

In [20]:
tokens_test= clean_text_1(test_text)
print(len(tokens_test))
print(len(set(tokens_test)))

5
5


In [21]:
# generating text

def generate_sequence(model, tokenizer, sequence_length, seed_text, n_words):
    out = list()
    in_text = seed_text
    # generate a fixed number of words
    for _ in range(n_words):
        # encode the text as integer
        encoded_text = tokenizer.texts_to_sequences([in_text])[0]
        # truncate sequences to a fixed length
        encoded_text = pad_sequences([encoded_text], maxlen=sequence_length, truncating='pre')
        # predict probabilities for each word
        yhat = np.argmax(model.predict(encoded_text, verbose=0), axis= -1)
        # map predicted word index to word
        out_word = ''
        for word, index in tokenizer.word_index.items():
            if index == yhat:
                out_word = word
                break
        # append to input
        in_text += ' ' + out_word
        out.append(out_word)
    return ' '.join(out)

In [ ]:
# generate new text
sequence_length = 10
text_generated = generate_sequence(model_load, tokenizer, sequence_length, tokens_test, 9)
print(text_generated)

breakers limbs black alien locality happened strictly racing might
